In [3]:
import pandas as pd
import numpy as np

In [5]:
# If you've saved the cleaned data earlier
data = pd.read_csv("cleaned_ecommerce_data.csv")
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_hour,event_day,event_weekday,event_month,event_date,main_category
0,2019-10-01 00:00:00+00:00,view,44600062,2103807459595387724,Unknown,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,0,1,1,10,2019-10-01,Unknown
1,2019-10-01 00:00:00+00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,0,1,1,10,2019-10-01,appliances
2,2019-10-01 00:00:01+00:00,view,17200506,2053013559792632471,furniture.living_room.sofa,Unknown,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,0,1,1,10,2019-10-01,furniture
3,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,0,1,1,10,2019-10-01,computers
4,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,0,1,1,10,2019-10-01,electronics


In [7]:
# Step 2: Create Target Variable — Purchase Label

# Mark session as 1 if it has a purchase event
session_labels = data.groupby('user_session')['event_type'].apply(lambda x: 1 if 'purchase' in x.values else 0)
session_labels = session_labels.rename('purchase_label')

In [9]:
# Optional: Optimize memory before aggregation
data['event_type'] = data['event_type'].astype('category')
data['brand'] = data['brand'].astype('category')
data['main_category'] = data['main_category'].astype('category')
# Skip user_session — too many unique values

# Use Boolean masks instead of lambda for speed
data['is_view'] = (data['event_type'] == 'view').astype('int8')
data['is_cart'] = (data['event_type'] == 'cart').astype('int8')
data['is_remove'] = (data['event_type'] == 'remove_from_cart').astype('int8')

# Perform groupby aggregation
session_features = data.groupby('user_session').agg(
    user_id=('user_id', 'first'),
    session_start=('event_time', 'min'),
    session_end=('event_time', 'max'),
    num_events=('event_type', 'count'),
    num_views=('is_view', 'sum'),
    num_carts=('is_cart', 'sum'),
    num_remove_from_cart=('is_remove', 'sum'),
    num_unique_products=('product_id', 'nunique'),
    num_unique_categories=('category_id', 'nunique'),
    avg_price=('price', 'mean'),
    max_price=('price', 'max'),
    min_price=('price', 'min'),
    num_brands=('brand', 'nunique'),
).reset_index()


In [13]:
# Step 4 – Feature engineering on time

# Ensure datetime format before subtraction
session_features['session_start'] = pd.to_datetime(session_features['session_start'])
session_features['session_end'] = pd.to_datetime(session_features['session_end'])
session_features['session_duration'] = (session_features['session_end'] - session_features['session_start']).dt.total_seconds()
session_features['hour_of_day'] = session_features['session_start'].dt.hour

In [15]:
# Step 5 – Merge features with labels

final_df = session_features.join(session_labels, how='left')
final_df.dropna(subset=['purchase_label'], inplace=True)
final_df.reset_index(inplace=True)

In [17]:
# Step 6 – Save final Phase 2 output

final_df.to_csv("session_features.csv", index=False)